# Modélisation d’un réseau BT – Trey
Par M. Buchser & M. Joye

Réalisé lors du semestre n°1 de 2024-2025



### Code of the work done

Import of librairies

In [1143]:
import numpy as np
import pandas as pd
import pandapower as pp
import pp_heig_plot as pp_plot
import pp_heig_simulation as pp_sim
from datetime import time

Import the net from the Excel file

In [1144]:
net_file_path = "data/trey_power_network.xlsx"
net = pp_sim.load_net_from_xlsx(file_path=net_file_path)
# net

Control that everything is okay

In [1145]:
# net.bus
# net.res_line
# net.ext_grid

Plot the net

In [1146]:
plot = pp_plot.plot_power_network(net, plot_title="Network of Trey")

Test power flow - plot with constant flow

In [1147]:
pp.runpp(net)
pp_plot.plot_powerflow_result(net=net, plot_title="Network of Trey")

In [1148]:
# net.res_bus

In [1149]:
# net.res_line

In [1150]:
# net.res_trafo

2.1.2.1 Identification and modeling of load curves

In [1151]:
# Import datas from CSV file
consumer_file_path = "data/liste_des_batiments_vf.csv"
consumer_data = pd.read_csv(consumer_file_path, encoding="latin1")

# Control
# consumer_data

2.1.2.2 Estimation of annual consumption using floor space

In [1152]:
# COP estimation for heat cumsuption with heat pump
COP = 4

# Create a mapping dictionary for consumption (SIA standard)
consumption_sia = {
    "one_housing": {
        "electricity": 17.8 + 0 + 4.2,
        "heating": 23.4,
        "hot_water": 13.5,
        "special_heating": 23.4 / COP,
        "special_hot_water": 13.5 / COP,
    },  # Individual housing
    "multi_housing": {
        "electricity": 21.6 + 0 + 4.2,
        "heating": 14.2,
        "hot_water": 16.9,
        "special_heating": 14.2 / COP,
        "special_hot_water": 16.9 / COP,
    },  # Collective housing
    "farm": {
        "electricity": 6.3 + 0 + 10.5,
        "heating": 11.5,
        "hot_water": 0.9,
        "special_heating": 11.5 / COP,
        "special_hot_water": 0.9 / COP,
    },  # Agricultural building (assimilated to warehouse)
    "church": {
        "electricity": 4.6 + 0 + 20.9,
        "heating": 6.4,
        "hot_water": 7.3,
        "special_heating": 6.4 / COP,
        "special_hot_water": 7.3 / COP,
    },  # Ecclesiastical building (assimilated to performance hall)
    "industial": {
        "electricity": 16.8 + 50 + 20.5,
        "heating": (9.0 + 10.7) / COP,
        "hot_water": 2.4 / COP,
    },  # Industrial building (heavy work)
}


# Function to estimate electricity consumption
def estimate_consumption(row):
    building_type = row["building_class"]
    surface = row["Empreinte au sol [m2]"]

    if building_type in consumption_sia:
        conso = consumption_sia[building_type]
        # Calculate electricity consumption (sum of appliances, installations, and lighting)
        electricity_consumption = conso["electricity"] * surface

        # Add thermal energy if heated electrically
        heating_consumption = 0
        if row["heat_source_1"] == "electricity":
            heating_consumption += conso["heating"] * surface
        elif row["heat_1"] == "PAC_1":
            heating_consumption += conso["special_heating"] * surface
            if row["heatwater_source_1"] == "unknown":
                heating_consumption += conso["special_hot_water"] * surface
        elif row["building_class"] == "industial":
            heating_consumption += (conso["heating"] + conso["hot_water"]) * surface
        if row["heatwater_source_1"] == "electricity":
            heating_consumption += conso["hot_water"] * surface

        return electricity_consumption + heating_consumption
    else:
        print(f"Building class {building_type} is missing.")


# Apply the function to each row
consumer_data["Estimated_Electricity_Consumption_kWh"] = consumer_data.apply(
    estimate_consumption, axis=1
)

In [1153]:
# # Display the results (verification)
# consumer_data[
#     [
#         "Cabinet",
#         "building_class",
#         "Empreinte au sol [m2]",
#         "Estimated_Electricity_Consumption_kWh",
#     ]
# ]

2.1.2.3 Definition of two scenarios of load curve

In [1154]:
import math

# Specify the file path
file_path = "data/Load_curves.xlsx"

# List of sheets to read
sheet_names = ["one_housing", "multi_housing", "farm", "church", "industial"]

# Dictionary to store data from each sheet
load_curves_df = {}

# Power factor chosen (page 5 of the low voltage course of M. Pellerin)
min_cos = 0.95

# Read the specified sheets
for sheet in sheet_names:
    load_curves = pd.read_excel(file_path, sheet_name=sheet)

    # Store the cleaned data
    load_curves_df[sheet] = load_curves

    # Normalize the 'winter' and 'summer' columns for each building type
for sheet, df in load_curves_df.items():
    # Calculate the correction factor for the 'winter' and 'summer' columns
    winter_corr_fact = df["winter"].sum() * 365 / (4 * 1000 * 1000)
    summer_corr_fact = df["summer"].sum() * 365 / (4 * 1000 * 1000)

    # Normalize the columns
    df["winter_p"] = df["winter"] / winter_corr_fact
    df["summer_p"] = df["summer"] / summer_corr_fact

    # Drop the old winter and summer column
    df = df.drop(["winter", "summer"], axis=1)

    # Adding the reactive power
    df["winter_q"] = df["winter_p"] * math.tan(math.acos(min_cos))
    df["summer_q"] = df["summer_p"] * math.tan(math.acos(min_cos))

    # Change order of the columns
    new_order = ["time", "winter_p", "winter_q", "summer_p", "summer_q"]
    df = df[new_order]

    # Store the normalized DataFrame back into the dictionary
    load_curves_df[sheet] = df

In [1155]:
# Verification of the 1000 kWh/year value
# for sheet, df in load_curves_df.items():
#     # Display the new total for verification
#     new_winter_sum = df["winter"].sum() * 365 / (4 * 1000)
#     new_summer_sum = df["summer"].sum() * 365 / (4 * 1000)
#     print(f"{sheet} - Normalized winter total: {new_winter_sum:.2f} kWh/year")
#     print(f"{sheet} - Normalized summer total: {new_summer_sum:.2f} kWh/year")

In [1156]:
# # Display a preview of the data with the sheet name as a title (verification)
# for sheet, df in load_curves_df.items():
#     print("=" * 30)  # Separator line
#     print(f"Data from sheet: {sheet}")  # Display the sheet name
#     print("=" * 30)
#     print(df.head())  # Display the first rows of the dataframe
#     print("\n")  # Add spacing for better readability

Definition of the two consumption curves on each cabinet

In [1157]:
# Dictionary to store the load curves for each cabinet
cabinet_load_curves_df = {}

W_TO_MW = 1e-6

# Loop through each row in consumer_data
for index, row in consumer_data.iterrows():
    building_class = row["building_class"]
    cabinet = row["Cabinet"]
    estimated_consumption = row["Estimated_Electricity_Consumption_kWh"]

    # Find the corresponding load curve for the building class
    if building_class in load_curves_df:
        load_curve = load_curves_df[building_class].copy()

        # Calculate the scaling factor
        scaling_factor = estimated_consumption / 1000 * W_TO_MW

        # Scale the summer and winter curves
        load_curve["winter_p"] *= scaling_factor
        load_curve["summer_p"] *= scaling_factor
        load_curve["winter_q"] *= scaling_factor
        load_curve["summer_q"] *= scaling_factor

        # Add this load curve to the dictionary under the cabinet's name
        if cabinet in cabinet_load_curves_df:
            # Only adding the curves, not the time
            cabinet_load_curves_df[cabinet]["winter_p"] += load_curve["winter_p"]
            cabinet_load_curves_df[cabinet]["summer_p"] += load_curve["summer_p"]
            cabinet_load_curves_df[cabinet]["winter_q"] += load_curve["winter_q"]
            cabinet_load_curves_df[cabinet]["summer_q"] += load_curve["summer_q"]

        else:
            # Create a new DataFrame for this cabinet
            cabinet_load_curves_df[cabinet] = load_curve

# Create an empty DataFrame to hold the combined data
load_curves_tot_df = pd.DataFrame()

# Loop through each cabinet in the load curve dictionary
for cabinet, load_curve in cabinet_load_curves_df.items():
    # Rename columns to include the cabinet name
    load_curve = load_curve.rename(
        columns={
            "winter_p": f"winter_p \n {cabinet}",
            "winter_q": f"winter_q \n {cabinet}",
            "summer_p": f"summer_p \n {cabinet}",
            "summer_q": f"summer_q \n {cabinet}",
        }
    )

    # Merge into the combined DataFrame on 'time'
    if load_curves_tot_df.empty:
        load_curves_tot_df = load_curve  # Initialize with the first cabinet's data
    else:
        load_curves_tot_df = pd.merge(
            load_curves_tot_df, load_curve, on="time", how="outer"
        )

# Save the combined data to an Excel file
output_file = "output/Out_Load_Curves.xlsx"
load_curves_tot_df.to_excel(output_file, index=False)

print(f"All cabinet load curves saved to {output_file}")

All cabinet load curves saved to output/Out_Load_Curves.xlsx


In [1158]:
# # Display the data for a specific cabinet (verification)
# for cabinet, df in cabinet_load_curves_df.items():
#     print(f"\nData for Cabinet: {cabinet}")
#     print(df.head())

Solar production on each cabinet

In [1159]:
# File path for the solar production curves
solar_file_path = "data/Solar_production_curves.xlsx"

# Read all sheet names from the solar production file
solar_sheet_names = pd.ExcelFile(solar_file_path).sheet_names

# Dictionary to store solar production curves
solar_curves_df = {}

# Read solar production curves for each cabinet
for sheet in solar_sheet_names:
    # Read the sheet
    solar_data = pd.read_excel(solar_file_path, sheet_name=sheet)

    # Rename columns for consistency
    solar_data.rename(
        columns={
            "Temps (heure:min)": "time",
            "Énergie [W] hiver": "winter_p",
            "Énergie [W] été": "summer_p",
        },
        inplace=True,
    )

    # Adding the reactive power
    solar_data["winter_q"] = 0
    solar_data["summer_q"] = 0

    # Scale the W curves to MW
    solar_data["winter_p"] *= W_TO_MW
    solar_data["summer_p"] *= W_TO_MW

    # Change order of the columns
    new_order = ["time", "winter_p", "winter_q", "summer_p", "summer_q"]
    solar_data = solar_data[new_order]

    # Convert 'time' column to match load curve format
    solar_data["time"] = pd.to_datetime(solar_data["time"], format="%H:%M:%S").dt.time

    # Store the solar curve in the dictionary
    solar_curves_df[sheet] = solar_data

import pandas as pd

# Create an empty DataFrame to hold the combined solar data
solar_curves_tot_df = pd.DataFrame()

# Loop through each cabinet in the solar curve dictionary
for cabinet, solar_curve in solar_curves_df.items():
    # Rename columns to include the cabinet name
    solar_curve = solar_curve.rename(
        columns={
            "winter_p": f"winter_p \n {cabinet}",
            "winter_q": f"winter_q \n {cabinet}",
            "summer_p": f"summer_p \n {cabinet}",
            "summer_q": f"summer_q \n {cabinet}",
        }
    )

    # Merge into the combined DataFrame on 'time'
    if solar_curves_tot_df.empty:
        solar_curves_tot_df = solar_curve  # Initialize with the first cabinet's data
    else:
        solar_curves_tot_df = pd.merge(
            solar_curves_tot_df, solar_curve, on="time", how="outer"
        )

# Save the combined solar data to an Excel file
output_file = "output/Out_Solar_Curves.xlsx"
solar_curves_tot_df.to_excel(output_file, index=False)

print(f"All solar production curves saved to {output_file}")

All solar production curves saved to output/Out_Solar_Curves.xlsx


In [1160]:
# # Display the updated load curve for a specific cabinet (verification)
# for cabinet, df in solar_curves_df.items():
#     print(f"\nSolar Production Curve for Cabinet: {cabinet}")
#     print(solar_curves_df[cabinet])

Choice between winter or summer profiles

In [1161]:
# Choose the season: 'winter' or 'summer'
# selected_season = "winter"
selected_season = "summer"

In [1162]:
from openpyxl import load_workbook

# File path for the Excel file
file_path_power_network = "data/trey_power_network_simu.xlsx"

# Profil mappings for winter and summer
winter_profiles_load = [8, 16, 18, 14, 2, 20, 6, 4, 12, 0, 10, 22]
summer_profiles_load = [9, 17, 19, 15, 3, 21, 7, 5, 13, 1, 11, 23]
winter_profiles_sgen = [0, 2]
summer_profiles_sgen = [1, 3]

# Load the Excel file
workbook_power_network = load_workbook(file_path_power_network)

# Modify the 'load' sheet
load_sheet = workbook_power_network["load"]

# Update the 'profile_mapping' column based on the selected season
if selected_season == "winter":
    profile_values_load = winter_profiles_load
    profile_values_sgen = winter_profiles_sgen
elif selected_season == "summer":
    profile_values_load = summer_profiles_load
    profile_values_sgen = summer_profiles_sgen
else:
    raise ValueError("Invalid season. Choose 'winter' or 'summer'.")

for row_idx, value in enumerate(
    profile_values_load, start=2
):  # Start from row 2 to skip headers and column 7 is profile_mapping column
    load_sheet.cell(row=row_idx, column=7).value = value

# Modify the 'sgen' sheet to exclude it from first simulation
sgen_sheet = workbook_power_network["sgen"]

for row in range(2, sgen_sheet.max_row + 1):
    sgen_sheet.cell(row=row, column=8).value = None

# Save the updated workbook for page sgen
workbook_power_network.save(file_path_power_network)
print(f"File {file_path_power_network} has been updated successfully.")

File data/trey_power_network_simu.xlsx has been updated successfully.


Import the power profiles from the Excel file

In [1163]:
power_profile_file_path = "data/power_profile.xlsx"
net_file_path = "data/trey_power_network_simu.xlsx"
time_series = pp_sim.load_power_profile_form_xlsx(file_path=power_profile_file_path)
net_trey = pp_sim.load_net_from_xlsx(file_path=net_file_path)

Apply the profile and the net

In [1164]:
pp_sim.apply_power_profile(
    net=net_trey, equipment="load", power_profiles=time_series["load"]
)

Simulation

In [1165]:
pp_sim.create_output_writer(
    net=net_trey, add_results=["res_bus.p_mw", "res_line.p_from_mw"]
)

# time_series["load"]

In [1166]:
result_df = pp_sim.run_time_simulation(net=net_trey)

/home/micael/miniforge3/lib/python3.12/site-packages/pandapower/timeseries/output_writer.py:177: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 96)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.



In [1167]:
# net.load

In [1168]:
pp_plot.plot_timeseries_result(
    data_df=result_df["res_line.loading_percent"],
    ylabel="Charge [%]",
    plot_title="Charge sur les lignes",
    filename="loading_result",
)

Adding the solar production to the simulation

In [1169]:
# Modify the 'sgen' sheet
sgen_sheet = workbook_power_network["sgen"]

for row_idx, value in enumerate(
    profile_values_sgen, start=2
):  # Start from row 2 to skip headers and column 8 is profile_mapping column
    sgen_sheet.cell(row=row_idx, column=8).value = value

# Save the updated workbook for page sgen
workbook_power_network.save(file_path_power_network)
print(f"File {file_path_power_network} has been updated successfully.")

# Update net with solar production
net_trey = pp_sim.load_net_from_xlsx(file_path=net_file_path)

pp_sim.create_output_writer(
    net=net_trey, add_results=["res_bus.p_mw", "res_line.p_from_mw"]
)
pp_sim.apply_power_profile(
    net=net_trey, equipment="load", power_profiles=time_series["load"]
)

pp_sim.apply_power_profile(
    net=net_trey, equipment="sgen", power_profiles=time_series["sgen"]
)

# time_series["sgen"]

File data/trey_power_network_simu.xlsx has been updated successfully.


In [1170]:
result_df = pp_sim.run_time_simulation(net=net_trey)
pp_plot.plot_timeseries_result(
    data_df=result_df["res_line.loading_percent"],
    ylabel="Charge [%]",
    plot_title="Charge sur les lignes",
    filename="loading_result",
)

/home/micael/miniforge3/lib/python3.12/site-packages/pandapower/timeseries/output_writer.py:177: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'range(0, 96)' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.



In [1171]:
pp_plot.plot_powerflow_result(net=net_trey, plot_title="Network of Trey")